# Macro research topic 1
## Quantifying geopolitical uncertainty that isn’t yet reflected in the markets


### Introduction

-  The world is currently experiencing more conflicts than at any time since the post-9/11 Iraq war.
-  I believe the risk of nuclear war is at its highest level since the Cold War.
-  Despite these risks, the S&P 500 keeps reaching new highs, driven by the ChatGPT initiated tech boom.
-  Markets seem aware of war risks, as shown by Bitcoin's quick drop when Iran attacked Israel with war drones. S&P 500 did not reflect this drop as it was a Sunday.
-  This leads me to believe that that markets believe AI's economic benefits outweigh the potential negative impacts of war.
-  This study aims to:
  1. Measure the positive effects of AI and negative effects of war on the economy
  2. Analyze how these 2 factors are currently priced into markets
  3. Compare the current situations with the 2000 tech boom and Cold War era to identify possible future scenarios.

In [5]:
import pandas as pd
from yahooquery import Ticker

tickers = [
    "^DJI",    # Dow Jones Industrial Average
    "^DJUS",   # Dow Jones U.S. Total Stock Market Index
    "^DWCF",   # Dow Jones Total Stock Market Index
    "CL=F",    # WTI Crude Oil
    "BZ=F",    # Brent Crude Oil
    "BTC-USD", # BTC
    "NVDA",    # NVIDIA
    "^GSPC",   # S&P 500
    "^TNX",    # 10-Year Treasury Yield
    "^TYX",    # 30-Year Treasury Yield
    "^FVX",    # 5-Year Treasury Yield
    "^IRX",    # 13-Week Treasury Bill Yield
]

for ticker in tickers:
    data = Ticker(ticker)
    history = data.history(period="max")
    print(f"Data for {ticker}:")
    print(history.head(2))
    print("\n")


/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


Data for ^DJI:
                          open         high          low        close  \
symbol date                                                             
^DJI   1992-01-02  3152.100098  3172.629883  3139.310059  3172.399902   
       1992-01-03  3172.399902  3210.639893  3165.919922  3201.500000   

                     volume     adjclose  
symbol date                               
^DJI   1992-01-02  23550000  3172.399902  
       1992-01-03  23620000  3201.500000  




/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


Data for ^DJUS:
                         open        high         low       close      volume  \
symbol date                                                                     
^DJUS  2000-02-14  327.290009  329.179993  322.260010  327.940002  15448600.0   
       2000-02-15  327.890015  331.609985  324.190002  330.339996  18067700.0   

                     adjclose  
symbol date                    
^DJUS  2000-02-14  327.940002  
       2000-02-15  330.339996  




/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


...

/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


Data for ^FVX:
                   open  high   low  close  volume  adjclose
symbol date                                                 
^FVX   1962-01-02  3.88  3.88  3.88   3.88     0.0      3.88
       1962-01-03  3.87  3.87  3.87   3.87     0.0      3.87




/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


Data for ^IRX:
                   open  high   low  close  volume  adjclose
symbol date                                                 
^IRX   1960-01-04  4.52  4.52  4.52   4.52     0.0      4.52
       1960-01-05  4.55  4.55  4.55   4.55     0.0      4.55


Data for ^TYX-^TNX:
Empty DataFrame
Columns: [high, low, volume, open, close]
Index: []


Data for USSFCT10:
Empty DataFrame
Columns: [high, low, volume, open, close]
Index: []


Data for UXYAISPS:
Empty DataFrame
Columns: [high, low, volume, open, close]
Index: []


Data for YIYUXY:
Empty DataFrame
Columns: [high, low, volume, open, close]
Index: []


Data for SRATEUSD6M.ME:
                                           open    high     low   close  \
symbol        date                                                        
SRATEUSD6M.ME 2018-07-27 23:54:59+03:00  1.1429  1.1429  1.1429  1.1429   

                                         volume  adjclose  
symbol        date                                         
SRATEUSD6M.ME 

/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
